In [1]:
import csv
import mysql.connector

SCHEMA_NAME = 'odmeta'
CAT_DS_NAME = 'catalog_dataset'
CAT_COL_NAME = 'catalog_column'

# generate and execute DROP TABLE for socrata ddg catalog tables  
#
def drop_catalog_tables(mycursor):
    ddl = "DROP TABLE IF EXISTS {}.{};\n".format(SCHEMA_NAME, CAT_DS_NAME)
    if mycursor is not None:
        mycursor.execute(ddl)
    ddl = "DROP TABLE IF EXISTS {}.{};\n".format(SCHEMA_NAME, CAT_COL_NAME)
    if mycursor is not None:
        mycursor.execute(ddl)
    return ddl
    
def create_catalog_tables(mycursor):
    ddl = "CREATE TABLE {}.{} (\n".format(SCHEMA_NAME, CAT_DS_NAME)
    ddl += "domain_id CHAR(4) NOT NULL,\n"
    ddl += "dataset_id CHAR(9) NOT NULL,\n"
    ddl += "name VARCHAR(255) NOT NULL,\n"
    ddl += "description TEXT NOT NULL,\n"
    ddl += "PRIMARY KEY(domain_id, dataset_id))\n"
    if mycursor is not None:
        mycursor.execute(ddl)
    print(ddl)
            
    ddl = "CREATE TABLE {}.{} (\n".format(SCHEMA_NAME, CAT_COL_NAME)
    ddl += "domain_id CHAR(4) NOT NULL,\n"
    ddl += "dataset_id VARCHAR(255) NOT NULL,\n"
    ddl += "field_name VARCHAR(255) NOT NULL,\n"
    ddl += "name VARCHAR(255) NOT NULL,\n"
    ddl += "datatype VARCHAR(255) NOT NULL,\n"
    ddl += "format VARCHAR(255) NOT NULL,\n"
    ddl += "description TEXT NOT NULL,\n"
    ddl += "PRIMARY KEY(domain_id, dataset_id,field_name))\n\n"
    if mycursor is not None:
        mycursor.execute(ddl)
    print(ddl)
    
    return ddl

In [2]:
print (drop_catalog_tables(None))

DROP TABLE IF EXISTS odmeta.catalog_column;



In [3]:
print (create_catalog_tables(None))

CREATE TABLE odmeta.catalog_dataset (
domain_id CHAR(4) NOT NULL,
dataset_id CHAR(9) NOT NULL,
name VARCHAR(255) NOT NULL,
description TEXT NOT NULL,
PRIMARY KEY(domain_id, dataset_id))

CREATE TABLE odmeta.catalog_column (
domain_id CHAR(4) NOT NULL,
dataset_id VARCHAR(255) NOT NULL,
field_name VARCHAR(255) NOT NULL,
name VARCHAR(255) NOT NULL,
datatype VARCHAR(255) NOT NULL,
format VARCHAR(255) NOT NULL,
description TEXT NOT NULL,
PRIMARY KEY(domain_id, dataset_id,field_name))


CREATE TABLE odmeta.catalog_column (
domain_id CHAR(4) NOT NULL,
dataset_id VARCHAR(255) NOT NULL,
field_name VARCHAR(255) NOT NULL,
name VARCHAR(255) NOT NULL,
datatype VARCHAR(255) NOT NULL,
format VARCHAR(255) NOT NULL,
description TEXT NOT NULL,
PRIMARY KEY(domain_id, dataset_id,field_name))




In [4]:
def insert_catalog_table(mydb, domain, ds):
    ddl = "INSERT INTO {}.{} ".format(SCHEMA_NAME, CAT_DS_NAME)
    ddl += "VALUES (%s,%s,%s,%s)\n"
    if mydb is not None:
        mydb._db_cur.execute(ddl, (domain, ds['id'],ds['name'],ds['description']))
        print( mydb._db_cur.statement)

    ddl = "INSERT INTO {}.{} ".format(SCHEMA_NAME, CAT_COL_NAME)
    ddl += "VALUES (%s,%s,%s,%s,%s,%s,%s)\n"
    print(ddl)
    # zip the column-related attributes into a single list
    col_count = len(ds['columns_name'])
    domain_ids = [domain] * col_count
    ds_ids = [ds['id']] * col_count
    col_formats = [str(col_format) for col_format in ds['columns_format']]
    cols = zip(domain_ids, ds_ids, ds['columns_field_name'], ds['columns_name'], ds['columns_datatype'],
               col_formats, ds['columns_description'])
    cols = list(cols)
    print("cols ", len(cols))
    print("row ", len(cols[0]))
    if mydb is not None:
        mydb._db_cur.executemany(ddl, cols)
        print(mydb._db_cur.statement)
        print (mydb._db_cur.rowcount, "Record inserted successfully")

In [5]:
import requests

domain = 'odde'

# Retrieve DDG catalog from socrata
# returns JSON
#
def fetch_catalog():
    resp = requests.get("http://api.us.socrata.com/api/catalog/v1?domains=data.delaware.gov")
    return resp.json()

catalog = fetch_catalog()
catalog['results'][1]

{'resource': {'name': 'Restaurant Inspection Violations',
  'id': '384s-wygj',
  'parent_fxf': None,
  'description': 'Because of a technical issue, the Department of Health and Social Services has not been able to update online restaurant inspection data since May 2017.  Since the Open Data Portal cannot provide current digital inspection data, we are removing the dataset from public view until this issue can be resolved.  The Division of Public Health is actively pursuing resolution of this issue through implementation of a new database system to house and manage the information, which is anticipated to be in place in 2018.  If you have any questions regarding a food inspection report conducted after May 2017, please contact the Office of Food Protection in the Division of Public Health at 302-744-4546.\r\n\r\n\r\n\r\nThis is a public use data file on Delaware restaurant inspections from January 1, 2016 through present. It includes a list of restaurants with violations reported durin

In [6]:
# rip out the datasets for convenience
datasets = [r for r in [x['resource'] for x in catalog['results']]
    if r['type'] == 'dataset']
datasets

for ds in datasets:
    print(insert_catalog_table(None, domain, ds))

INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  13
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  33
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  14
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  34
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  79
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  10
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  15
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  8
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  8
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  7
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  10
row  7
None
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,

In [7]:
class MyDB:
    def __init__(self):
        self._db_connection  = mysql.connector.connect(
            host='127.0.0.1',
            user='oddeuser',
            password='oddepass',
            db='odmeta',
            autocommit=True)
        self._db_cur = self._db_connection.cursor(prepared=True)

    def query(self, query, params=''):
        return self._db_cur.execute(query, params)

    def __del__(self):
        self._db_connection.close()
        
mydb = MyDB()

In [8]:
print(drop_catalog_tables(mydb._db_cur))


DROP TABLE IF EXISTS odmeta.catalog_column;



In [9]:
print(create_catalog_tables(mydb._db_cur))


CREATE TABLE odmeta.catalog_dataset (
domain_id CHAR(4) NOT NULL,
dataset_id CHAR(9) NOT NULL,
name VARCHAR(255) NOT NULL,
description TEXT NOT NULL,
PRIMARY KEY(domain_id, dataset_id))

CREATE TABLE odmeta.catalog_column (
domain_id CHAR(4) NOT NULL,
dataset_id VARCHAR(255) NOT NULL,
field_name VARCHAR(255) NOT NULL,
name VARCHAR(255) NOT NULL,
datatype VARCHAR(255) NOT NULL,
format VARCHAR(255) NOT NULL,
description TEXT NOT NULL,
PRIMARY KEY(domain_id, dataset_id,field_name))


CREATE TABLE odmeta.catalog_column (
domain_id CHAR(4) NOT NULL,
dataset_id VARCHAR(255) NOT NULL,
field_name VARCHAR(255) NOT NULL,
name VARCHAR(255) NOT NULL,
datatype VARCHAR(255) NOT NULL,
format VARCHAR(255) NOT NULL,
description TEXT NOT NULL,
PRIMARY KEY(domain_id, dataset_id,field_name))




In [10]:
mydb._db_connection.get_warnings = True
for ds in datasets:
    insert_catalog_table(mydb, domain, ds)



INSERT INTO odmeta.catalog_dataset VALUES (%s,%s,%s,%s)
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  13
row  7
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)
13 Record inserted successfully
INSERT INTO odmeta.catalog_dataset VALUES (%s,%s,%s,%s)
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  33
row  7
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)
33 Record inserted successfully
INSERT INTO odmeta.catalog_dataset VALUES (%s,%s,%s,%s)
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  14
row  7
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)
14 Record inserted successfully
INSERT INTO odmeta.catalog_dataset VALUES (%s,%s,%s,%s)
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  34
row  7
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)
34 Record inserted successfully
INSERT INTO odmeta.catalog_dataset VALUES (%s,%s,%s,%s)
INSERT INTO 

INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)
10 Record inserted successfully
INSERT INTO odmeta.catalog_dataset VALUES (%s,%s,%s,%s)
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  11
row  7
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)
11 Record inserted successfully
INSERT INTO odmeta.catalog_dataset VALUES (%s,%s,%s,%s)
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  13
row  7
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)
13 Record inserted successfully
INSERT INTO odmeta.catalog_dataset VALUES (%s,%s,%s,%s)
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  13
row  7
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)
13 Record inserted successfully
INSERT INTO odmeta.catalog_dataset VALUES (%s,%s,%s,%s)
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)

cols  18
row  7
INSERT INTO odmeta.catalog_column VALUES (%s,%s,%s,%s,%s,%s,%s)
18 R